### Read Public Groups/Pages classify groups by State


In [1]:
import re
import pandas as pd
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [2]:
def clean_address(row):
    name =  row.lower().split('/')
    if  row.lower().startswith('https') or  row.lower().startswith('http'):       
        return name[3]
    elif row.lower().startswith('www'):
        return name[1] 
    elif row.lower().startswith('facebook'):
        return name[1]
    else:
        print("Not found", row)
        return None

## 1. Reads  Info
Reads the information from all the posts from the studied period

In [3]:

path = "cscwDocs/parties/FBPagesGroupsData/FBPagesGroupsCities/"
posts1 = pd.read_csv(path+"LatinosEstadosPagesAug182020-Nov032020.csv")
#We rename the colum Page Name to be able to concat with Group Posts
posts1.rename(columns={'Page Name':'Group Name'}, inplace=True)
posts2 = pd.read_csv(path+"LatinosEstadosGroups-Aug182020-Aug252020.csv")
posts3 = pd.read_csv(path+"LatinosEstadosGroups-Aug262020-Aug312020.csv")
posts4 = pd.read_csv(path+"LatinosEstadosGroups-Sep012020-Sep042020.csv")
posts5 = pd.read_csv(path+"LatinosEstadosGroups-Sep052020-Sep082020.csv")
posts6 = pd.read_csv(path+"LatinosEstadosGroups-Sep092020-Sep122020.csv")
posts7 = pd.read_csv(path+"LatinosEstadosGroups-Sep132020-Sep162020.csv")
posts8 = pd.read_csv(path+"LatinosEstadosGroups-Sep172020-Sep202020.csv")
posts9 = pd.read_csv(path+"LatinosEstadosGroups-Sep212020-Sep242020.csv")
posts10 = pd.read_csv(path+"LatinosEstadosGroups-Sep252020-Sep282020.csv")
posts11 = pd.read_csv(path+"LatinosEstadosGroups-Sep292020-Oct022020.csv")
posts12 = pd.read_csv(path+"LatinosEstadosGroups-Oct032020-Oct062020.csv")
posts13 = pd.read_csv(path+"LatinosEstadosGroups-Oct072020-Oct102020.csv")
posts14 = pd.read_csv(path+"LatinosEstadosGroups-Oct112020-Oct142020.csv")
posts15 = pd.read_csv(path+"LatinosEstadosGroups-Oct152020-Oct172020.csv")
posts16 = pd.read_csv(path+"LatinosEstadosGroups-Oct182020--Oct212020.csv")
posts17 = pd.read_csv(path+"LatinosEstadosGroups-Oct222020-Oct252020.csv")
posts18 = pd.read_csv(path+"LatinosEstadosGroups-Oct262020-Oct292020.csv")
posts19 = pd.read_csv(path+"LatinosEstadosGroups-October302020-Nov032020.csv")
posts = pd.concat([posts1,posts2,posts3,posts4,posts5,posts6,posts7,posts8,posts9,
                   posts10,posts11,posts12,posts13,posts14,posts15,posts16,posts17,posts18,posts19])
posts.shape

FileNotFoundError: [Errno 2] No such file or directory: 'cscwDocs/parties/FBPagesGroupsData/FBPagesGroupsCities/LatinosEstadosPagesAug182020-Nov032020.csv'

In [ ]:
pd.set_option('display.max_colwidth', 400)
posts['URL'].head()

In [ ]:
posts.head(3)

## 9. Filter the most relevant columns
### Add column to concat Message, Description, Link Text

In [ ]:
concat_message = posts['Message'].fillna('') + (' ' + posts['Description']).fillna('')+ (' ' + posts['Link Text']).fillna('') + (' ' + posts['Link']).fillna('')
posts['MessageDescr'] = concat_message

concat_just_message = posts['Message'].fillna('') + (' ' + posts['Description']).fillna('')+ (' ' + posts['Link Text']).fillna('')
posts['MessageOnly'] = concat_just_message

In [ ]:
posts.columns

In [ ]:
posts[posts['Link Text'].str.lower().str.contains("í", na = False)].head(1)

In [ ]:
#!/usr/bin/env python
import re

text = u'This is a smiley face \U0001f602'
print(text) # with emoji

def deEmojify(text):
    
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                          
                           "]+", flags = re.UNICODE)
    
    
    return regrex_pattern.sub(r'',text)

print(deEmojify(text))

def deEmojify(inputString):
    return inputString.encode('ascii', 'ignore').decode('ascii')

In [ ]:

posts['Group Name']=posts['Group Name'].apply(deEmojify)
posts['Group Name']=posts['Group Name'].apply(lambda i: i.replace('"', ''))
posts['Group Name']=posts['Group Name'].apply(lambda i: i.replace(',', ''))
posts['Facebook Id'] = posts['Facebook Id'].astype(int)

## Read FB Group Names & State


In [ ]:
fb_groups = pd.read_csv("cscwDocs/parties/FBGroupsPolitical_cities.csv")
fb_groups['Facebook Id'] = fb_groups['Page or Account URL'].str.strip("https://www.facebook.com/groups/")
fb_groups

### 4. US states

In [ ]:
us_states = pd.DataFrame({'State':['Alabama','Alaska','Arizona','Arkansas','California','Colorado','Connecticut','Delaware','District of Columbia','Florida',
             'Georgia','Hawaii','Idaho','Illinois','Indiana','Iowa','Kansas','Kentucky','Louisiana','Maine','Maryland','Massachusetts',
             'Michigan','Minnesota','Mississippi','Missouri','Montana','Nebraska','Nevada','New Hampshire','New Jersey','New Mexico',
             'New York','North Carolina','North Dakota','Ohio','Oklahoma','Oregon','Pennsylvania','Rhode Island','South Carolina',
             'South Dakota','Tennessee','Texas','Utah','Vermont','Virginia','Washington','West Virginia','Wisconsin','Wyoming']})

##  Count posts in general by state

In [ ]:
from functools import reduce
data_frames = [fb_groups, posts]
fb_groups['Facebook Id'] = fb_groups['Facebook Id'].astype(int)
df_posts_state  = reduce(lambda  left,right: pd.merge(left,right,on=['Facebook Id'],
                                            how='outer'), data_frames)
df_count = df_posts_state.groupby(['State']).agg({'Facebook Id':'count'}).reset_index() 

df_count.columns = ['State','Posts_Counts']
df_count.sort_values(by=['State'], ascending=True,inplace=True)
df_count.fillna(0, inplace=True)
df_count['Posts_Counts'] = df_count['Posts_Counts'].astype(int)
df_count.head(10)

In [ ]:
data_frames = [df_count, us_states]
missing_states  = reduce(lambda  left,right: pd.merge(left,right,on=['State'],
                                            how='outer'), data_frames)
missing_states.fillna(0, inplace=True)
missing_states['Posts_Counts'] = missing_states['Posts_Counts'].astype(int)
missing_states

In [ ]:
df_count.to_csv('cscwDocs/FB_groups_States/FB_groups_posts_by_state2.csv',index=False,na_rep=0)

In [ ]:
df_posts_state.columns

## 3. Which groups are publishing the majority of the content

In [ ]:
group = posts.groupby(['Group Name'])
final_groups = group.agg({'Facebook Id':'count'}).sort_values(by='Facebook Id',ascending=False)
final_groups

In [ ]:
final_groups.to_csv("cscwDocs/FB_groups_States/Posts_Per_group2.csv")

## 4. Filter posts per topic

In [ ]:
#Topics
economy = 'economy|economía|economia|welfare|taxes|unemployment|employment|job|minimum-wage|union|poverty|poor|homeless|stock|gdp'
health_care = 'health care|salud|medico|médico|salud|doctor|medicaid|medicare|planned parenthood|insurance|injury|disability'
coronavirus = 'coronavirus|covid|pandemia|vacuna|tos|fiebre|enfermo|pandemic|vaccine|fever|sick'
crime = 'crime|crimen|pandilla|violento|vandalism|violence|crime|robbery|theft|gang'
supreme_court = 'supreme court|suprema corte|juez|corte suprema|ruth bader ginsberg'
climate_change = 'climate change|cambio climático|environment|recycle|pollution|litter|nature|climate change|global warming '
immigration = 'immigration|migration|inmigración|migración|immigracion|indocumentado|frontera|pared|ice|border'
immigration2 = 'immigration|migration|inmigración|migración|immigracion|indocumentado|frontera|muro|ice|border|frontera'
gun_policy = 'gun policy|gun policies|política de armas|guns rights|NRA|armas|arma|pistola|política de armas|armas|gun violence|shooting|2nd amendment|second amendment|concealed carry'
foreign_policy ='foreign policy|política exterior|foreign policies|china|russia|united kingdom|england|mexico|canada|trade|embargo|ambassador|consulate|middle east|prime minister|war'
abortion = 'aborto|abortion|aborto|feto|embarazada|planned parenthood|abortion|fetus|trimester|miscarriage|pregnancy'
racism = 'racial inequality|etnic inqueality|racism|racismo|inequidad racial|inequidad étnica|crimen de odio|igualdad de raza y étnica|discriminación|black|hate crime|racism|desigualdad|blm|all lives matter'


pd.set_option('display.max_colwidth', 400)
path = 'cscwDocs/FB_groups_States/Topics/'
prefix = 'FB_groups_States_'

In [ ]:
posts_economy = df_posts_state.copy()
posts_economy = df_posts_state[df_posts_state['MessageDescr'].str.lower().str.contains(economy   ,na = False)]
print('economy', posts_economy.shape)
posts_economy.to_csv(path + prefix + 'economy' + '.csv')

In [ ]:
topic='health_care'
df = df_posts_state.copy()
df = df_posts_state[df_posts_state['MessageDescr'].str.lower().str.contains(health_care ,na = False)]
print(topic, df.shape)
df.to_csv(path + prefix + topic + '.csv')

In [ ]:
topic='coronavirus'
df = df_posts_state.copy()
df = df_posts_state[df_posts_state['MessageDescr'].str.lower().str.contains(coronavirus ,na = False)]
print(topic, df.shape)
df.to_csv(path + prefix + topic + '.csv')

In [ ]:
topic='crime'
df = df_posts_state.copy()
df = df_posts_state[df_posts_state['MessageDescr'].str.lower().str.contains(crime ,na = False)]
print(topic, df.shape)
df.to_csv(path + prefix + topic + '.csv')

In [ ]:
topic='supreme_court'
df = df_posts_state.copy()
df = df_posts_state[df_posts_state['MessageDescr'].str.lower().str.contains(supreme_court ,na = False)]
print(topic, df.shape)
df.to_csv(path + prefix + topic + '.csv')

In [ ]:
topic='climate_change'
df = df_posts_state.copy()
df = df_posts_state[df_posts_state['MessageDescr'].str.lower().str.contains(climate_change ,na = False)]
print(topic, df.shape)
df.to_csv(path + prefix + topic + '.csv')

In [ ]:
topic='immigration'
df = df_posts_state.copy()
df = df_posts_state[df_posts_state['MessageDescr'].str.lower().str.contains(immigration ,na = False)]
print(topic, df.shape)
df.to_csv(path + prefix + topic + '.csv')

In [ ]:
topic='immigration2'
df = df_posts_state.copy()
df = df_posts_state[df_posts_state['MessageDescr'].str.lower().str.contains(immigration2 ,na = False)]
print(topic, df.shape)
df.to_csv(path + prefix + topic + '.csv')

In [ ]:
topic='gun_policy'
df = df_posts_state.copy()
df = df_posts_state[df_posts_state['MessageDescr'].str.lower().str.contains(gun_policy ,na = False)]
print(topic, df.shape)
df.to_csv(path + prefix + topic + '.csv')

In [ ]:
topic='foreign_policy'
df = df_posts_state.copy()
df = df_posts_state[df_posts_state['MessageDescr'].str.lower().str.contains(foreign_policy ,na = False)]
print(topic, df.shape)
df.to_csv(path + prefix + topic + '.csv')

In [ ]:
topic='abortion'
df = df_posts_state.copy()
df = df_posts_state[df_posts_state['MessageDescr'].str.lower().str.contains(abortion ,na = False)]
print(topic, df.shape)
df.to_csv(path + prefix + topic + '.csv')

In [ ]:
topic='racism'
df = df_posts_state.copy()
df = df_posts_state[df_posts_state['MessageDescr'].str.lower().str.contains(racism ,na = False)]
print(topic, df.shape)
df.to_csv(path + prefix + topic + '.csv')

In [ ]:
topic='immigration2'
df = df_posts_state.copy()
df = df_posts_state[df_posts_state['MessageDescr'].str.lower().str.contains(immigration2 ,na = False)]
print(topic, df.shape)


In [ ]:
sample = df.sample(round(df.shape[0] * 0.10))
sample.shape

In [ ]:
print(topic, df.shape, sample.shape)
name='immigration_random'
sample.to_csv(path + prefix + name + '.csv')